In [1]:
import sys
import pandas as pd
import numpy as np
from numpy import sort
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
import scipy.cluster
from numpy import absolute, mean, sort, std
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
from scipy.stats import spearmanr

import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

from sklearn import datasets, metrics, preprocessing, model_selection
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold,RepeatedKFold, cross_val_score, cross_validate, cross_val_predict, GridSearchCV, RandomizedSearchCV, validation_curve, learning_curve
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error, max_error

import skopt
from skopt import BayesSearchCV 

from missingpy import MissForest

import shap
from BorutaShap import BorutaShap

import xgboost
import lightgbm
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier, StackingClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import filterwarnings
filterwarnings("ignore")

import matplotlib.pyplot as plt
import missingno as msno

seed = 0

data = pd.read_csv("training_cleaned.csv", header=0, sep=",")

data["BPlabel_encoded"] = data["BPlabel"].map(
    {"most likely": 1, "probable": 2, "possible":3,"least likely": 4}
)
Y = data["BPlabel_encoded"]
data = data.drop(["BPlabel"], 1)


xgbr = xgboost.XGBClassifier(random_state=seed, objective='reg:squarederror', verbosity = 0, eval_metric='mlogloss') 
xgbr_params = {
    'max_depth':  (1, 4), 
    'learning_rate': (0.01, 0.2, 'log-uniform'),  
    'n_estimators':  (10, 50), 
    'reg_alpha':  (1, 10, 'log-uniform'), 
    'reg_lambda':  (1, 10, 'log-uniform')} 

lgbm = LGBMClassifier(random_state=seed)
lgbm_params = {
    "max_depth": (1, 4),
    "learning_rate": (0.01, 0.2, "log-uniform"),
    "n_estimators": (10, 50),
    "reg_alpha": (1, 10, "log-uniform"),
    "reg_lambda": (1, 10, "log-uniform"),
}

catboost = CatBoostClassifier(random_seed=seed, verbose=False)
cat_params = {
     "iterations": (10, 50),
     'learning_rate': (0.01, 0.2, 'log-uniform'), 
     'depth':  (1, 4), 
}


gbr = GradientBoostingClassifier(random_state=seed)
gbr_params = {
    'learning_rate': (0.01, 0.2),
    'max_depth': (1, 4),
    "max_features":["log2","sqrt", "auto"],
    "criterion": ["friedman_mse", "mse"],
    'n_estimators': (10, 50)
    }

rfr = RandomForestClassifier(random_state=seed)
rfr_params={'n_estimators': (10, 50), 
             'max_features': ['sqrt', 'log2'],
             'max_depth' : (1, 4),
             'criterion' :['gini', 'entropy']} 

dt = DecisionTreeClassifier(random_state=seed)
dt_params= {"criterion": ['gini', 'entropy'],
            'max_features': ['sqrt', 'log2'],
            'max_depth' : (1, 4)}

extra = ExtraTreesClassifier(random_state=seed)
extra_params ={'n_estimators': (10, 50), 
             'max_features': ['sqrt', 'log2'],
             'max_depth' : (1, 4),
             'criterion' :['gini', 'entropy']}



knn = KNeighborsClassifier()
knn_params = {
    'n_neighbors':[7,9,11,13,15,17],
    'weights' : ['uniform','distance'],
    'metric' : ['euclidean','manhattan', 'minkowski']}


lr = LogisticRegression(penalty='l1', solver='liblinear',multi_class='auto',random_state=seed)
lr_params= {
    'penalty':['l1', 'l2'], 
    'C': [0.5, 1, 5, 10], 
    'max_iter':[500, 1000, 2500]}

svc = SVC()
svc_params = {
    'kernel': ['rbf'],
   'C': (1e0, 1e3),
   'gamma': ['scale', 'auto']}

inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)


results = []
names = []
medians =[]
models_list_balancedac = []
scoring = ['accuracy', 'balanced_accuracy', 'f1_weighted', 
          'precision_weighted','recall_weighted']



X = pd.read_csv("selected_features_training_data.csv", header=0)

X.columns = [
    regex.sub("_", col) if any(x in str(col) for x in set(("[", "]", "<"))) else col
    for col in X.columns.values
]

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0
)

df3 = pd.DataFrame(data= X, columns= X.columns)
df3.columns = [
    regex.sub("_", col) if any(x in str(col) for x in set(("[", "]", "<"))) else col
    for col in  X.columns.values
]
X_importance = X_test


models = []

models.append(('XGBR', BayesSearchCV(xgbr, xgbr_params, cv=inner_cv,iid=False,n_jobs=1, random_state=seed))) 
models.append(("LGBM", BayesSearchCV(lgbm, lgbm_params, cv=inner_cv, iid=False, n_jobs=1, random_state=seed)))
models.append(("CB", BayesSearchCV(catboost, cat_params, cv=inner_cv, iid=False, n_jobs=1, random_state=seed)))
models.append(('GBR', BayesSearchCV(gbr, gbr_params, cv=inner_cv,iid=False, n_jobs=1, random_state=seed)))
models.append(('RFR', BayesSearchCV(rfr, rfr_params, cv=inner_cv,iid=False, n_jobs=1, random_state=seed)))
models.append(('DT', BayesSearchCV(dt, dt_params, cv=inner_cv, iid=False, n_jobs=1, random_state=seed)))
models.append(('ExtraTrees', BayesSearchCV(extra, extra_params, cv=inner_cv, iid=False, n_jobs=1, random_state=seed)))


In [3]:
for name, model in models:
    nested_cv_results = model_selection.cross_validate(model, X , Y, cv=outer_cv, scoring=scoring, error_score="raise")
    nested_cv_results2 = model_selection.cross_val_score(model, X , Y, cv=outer_cv, scoring='balanced_accuracy', error_score="raise")
    results.append(nested_cv_results2)
    names.append(name)
    print(name, 'Nested CV results for all scores:', '\n', nested_cv_results, '\n')
    print(name, 'Accuracy Nested CV Average', np.median(nested_cv_results['test_accuracy']))
    print(name, 'Balanced Accuracy Nested CV Average', np.median(nested_cv_results['test_balanced_accuracy'] ))
    print(name, 'F1 Nested CV Average', np.median(nested_cv_results['test_f1_weighted'] ))
    print(name, 'Precision Nested CV Average', np.median(nested_cv_results['test_precision_weighted'] ))
    print(name, 'Recall Nested CV Average', np.median(nested_cv_results['test_recall_weighted'] ))
    model.fit(X, Y)
    print('\n')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best Estimator:", model.best_estimator_)
    best_model = model.best_estimator_
    print('\n')
    best_model.fit(X_train, Y_train)
    y_pred_train = best_model.predict(X_train)
    y_pred = best_model.predict(X_test)
    best_model.fit(X, Y)
    median_balancedac = np.median(nested_cv_results['test_balanced_accuracy'])
    models_list_balancedac.append((best_model, median_balancedac))


print('All results:', results)         

best_model1, best_balancedac = sorted(models_list_balancedac, key = lambda x: x[1], reverse=True)[0]
print('Best model by median balanced accuracy:',best_model1)

XGBR Nested CV results for all scores: 
 {'fit_time': array([250.45499706, 216.87853527, 202.06913996, 238.65334702,
       251.4058938 , 219.38489294, 259.57282495, 274.15962505,
       270.63423085, 246.43836784]), 'score_time': array([0.00625896, 0.00676608, 0.006814  , 0.00602484, 0.00634122,
       0.00619698, 0.00629401, 0.00607109, 0.00648594, 0.00597525]), 'test_accuracy': array([0.65789474, 0.63157895, 0.57894737, 0.55263158, 0.71052632,
       0.63157895, 0.47368421, 0.51351351, 0.62162162, 0.54054054]), 'test_balanced_accuracy': array([0.62777778, 0.6       , 0.48888889, 0.45      , 0.65      ,
       0.56736111, 0.41458333, 0.40267857, 0.55347222, 0.52222222]), 'test_f1_weighted': array([0.64205786, 0.5936494 , 0.52869306, 0.48987854, 0.68280675,
       0.60134628, 0.44278967, 0.47205577, 0.58139698, 0.49841017]), 'test_precision_weighted': array([0.67759543, 0.615311  , 0.55723684, 0.46710526, 0.70728406,
       0.59896617, 0.44402919, 0.49081081, 0.61577162, 0.46616541]),

In [4]:
results = []
names = []
othermodels = []

othermodels.append(('KNN', BayesSearchCV(knn, knn_params, cv=inner_cv, iid=False, n_jobs=1)))
othermodels.append(('SVC', BayesSearchCV(svc, svc_params, cv=inner_cv, iid=False, n_jobs=1)))
othermodels.append(('LR', BayesSearchCV(lr, lr_params, cv=inner_cv, iid=False, n_jobs=1)))

X2 = MinMaxScaler().fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(
    X2, Y, test_size=0.2, random_state=seed)

df3 = pd.DataFrame(data= X2, columns= X.columns)
df3.columns = [
    regex.sub("_", col) if any(x in str(col) for x in set(("[", "]", "<"))) else col
    for col in  X.columns.values
]
X_importance = df3

for name, model in othermodels:
    nested_cv_results = model_selection.cross_validate(model, X2 , Y, cv=outer_cv, scoring=scoring, error_score="raise")
    nested_cv_results2 = model_selection.cross_val_score(model, X2 , Y, cv=outer_cv, scoring='balanced_accuracy', error_score="raise")
    results.append(nested_cv_results2)
    names.append(name)
    print(name, 'Nested CV results for all scores:', '\n', nested_cv_results, '\n')
    print(name, 'Accuracy Nested CV Average', np.median(nested_cv_results['test_accuracy']))
    print(name, 'Balanced Accuracy Nested CV Average', np.median(nested_cv_results['test_balanced_accuracy'] ))
    print(name, 'F1 Nested CV Average', np.median(nested_cv_results['test_f1_weighted'] ))
    print(name, 'Precision Nested CV Average', np.median(nested_cv_results['test_precision_weighted'] ))
    print(name, 'Recall Nested CV Average', np.median(nested_cv_results['test_recall_weighted'] ))
    model.fit(X2, Y)
    print('\n')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best Estimator:", model.best_estimator_)
    best_model = model.best_estimator_
    print('\n')
    best_model.fit(X_train, Y_train)
    y_pred_train = best_model.predict(X_train)
    y_pred = best_model.predict(X_test)
    best_model.fit(X2, Y)
    median_balancedac = np.median(nested_cv_results['test_balanced_accuracy'])
    models_list_balancedac.append((best_model, median_balancedac))


print('All results:', results)   


best_model1, best_balancedac = sorted(models_list_balancedac, key = lambda x: x[1], reverse=True)[0]
print('Best model by median balanced accuracy:', best_model1)


KNN Nested CV results for all scores: 
 {'fit_time': array([59.88663888, 63.73879576, 68.38802409, 60.20594692, 63.12526417,
       63.49519682, 70.21255207, 70.45793295, 63.79241109, 73.39909196]), 'score_time': array([0.00652909, 0.00654817, 0.00686884, 0.00753212, 0.00664306,
       0.00684118, 0.00684905, 0.00632405, 0.00675488, 0.00529814]), 'test_accuracy': array([0.42105263, 0.57894737, 0.55263158, 0.44736842, 0.44736842,
       0.63157895, 0.44736842, 0.51351351, 0.48648649, 0.51351351]), 'test_balanced_accuracy': array([0.4       , 0.53333333, 0.47222222, 0.37222222, 0.42777778,
       0.59722222, 0.36458333, 0.45535714, 0.42152778, 0.48680556]), 'test_f1_weighted': array([0.36184211, 0.53292992, 0.49890266, 0.36687307, 0.41242236,
       0.61904762, 0.38840156, 0.4887764 , 0.4499667 , 0.49115418]), 'test_precision_weighted': array([0.32631579, 0.54094751, 0.51076555, 0.3132964 , 0.40023135,
       0.62030075, 0.3622291 , 0.55610996, 0.43243243, 0.47690418]), 'test_recall_weig

In [3]:
xgb =  xgboost.XGBClassifier(learning_rate=0.15120340125789705, n_estimators=50, max_depth=4, random_state=0, reg_alpha=3, reg_lambda=10)

lgbm =  LGBMClassifier(learning_rate=0.2, max_depth=1, n_estimators=50, random_state=0,
               reg_alpha=1, reg_lambda=1)

cb = CatBoostClassifier(depth=4, iterations=50, learning_rate=0.1964188294579477, random_seed=seed, verbose=False)

gb = GradientBoostingClassifier(learning_rate=0.19101624063861414, max_depth=1,
                           max_features='auto', n_estimators=47,
                           random_state=0)
                       
rf = RandomForestClassifier(criterion='entropy', max_depth=4, max_features='log2',
                       n_estimators=24, random_state=0)

dt = DecisionTreeClassifier(max_depth=3, max_features='sqrt', random_state=0)

et = ExtraTreesClassifier(criterion='entropy', max_depth=4, max_features='log2',
                     n_estimators=10, random_state=0)

knn = KNeighborsClassifier(metric='manhattan', n_neighbors=7)

svc = SVC(C=999.9984985741586)

lr = LogisticRegression(max_iter=1000, penalty='l1', random_state=0,
                   solver='liblinear')

In [4]:
target_names = ['most likely', 'probable', 'possible','least likely']
xgb.fit(X_train, Y_train)
predictions = list(xgb.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

[09:29:42] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

 most likely       0.33      0.10      0.15        10
    probable       0.55      0.81      0.66        27
    possible       0.33      0.19      0.24        16
least likely       0.88      0.91      0.89        23

    accuracy                           0.62        76
   macro avg       0.52      0.50      0.49        76
weighted avg       0.57      0.62      0.57        76



In [5]:
lgbm.fit(X_train, Y_train)
predictions = list(lgbm.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

 most likely       0.17      0.10      0.12        10
    probable       0.50      0.78      0.61        27
    possible       0.50      0.19      0.27        16
least likely       0.82      0.78      0.80        23

    accuracy                           0.57        76
   macro avg       0.50      0.46      0.45        76
weighted avg       0.55      0.57      0.53        76



In [6]:
cb.fit(X_train, Y_train)
predictions = list(cb.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

 most likely       0.00      0.00      0.00        10
    probable       0.50      0.70      0.58        27
    possible       0.27      0.19      0.22        16
least likely       0.79      0.83      0.81        23

    accuracy                           0.54        76
   macro avg       0.39      0.43      0.40        76
weighted avg       0.47      0.54      0.50        76



In [7]:
gb.fit(X_train, Y_train)
predictions = list(gb.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

 most likely       0.17      0.10      0.12        10
    probable       0.49      0.70      0.58        27
    possible       0.33      0.12      0.18        16
least likely       0.80      0.87      0.83        23

    accuracy                           0.55        76
   macro avg       0.45      0.45      0.43        76
weighted avg       0.51      0.55      0.51        76



In [8]:
rf.fit(X_train, Y_train)
predictions = list(rf.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

 most likely       0.00      0.00      0.00        10
    probable       0.51      0.85      0.64        27
    possible       0.33      0.06      0.11        16
least likely       0.84      0.91      0.87        23

    accuracy                           0.59        76
   macro avg       0.42      0.46      0.40        76
weighted avg       0.51      0.59      0.51        76



In [9]:
dt.fit(X_train, Y_train)
predictions = list(dt.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

 most likely       0.00      0.00      0.00        10
    probable       0.44      0.89      0.59        27
    possible       0.00      0.00      0.00        16
least likely       0.73      0.70      0.71        23

    accuracy                           0.53        76
   macro avg       0.29      0.40      0.33        76
weighted avg       0.38      0.53      0.43        76



In [10]:
et.fit(X_train, Y_train)
predictions = list(et.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

 most likely       0.25      0.10      0.14        10
    probable       0.45      0.89      0.60        27
    possible       0.00      0.00      0.00        16
least likely       0.74      0.61      0.67        23

    accuracy                           0.51        76
   macro avg       0.36      0.40      0.35        76
weighted avg       0.42      0.51      0.43        76



In [11]:
knn.fit(X_train, Y_train)
predictions = list(knn.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

 most likely       0.00      0.00      0.00        10
    probable       0.35      0.44      0.39        27
    possible       0.14      0.06      0.09        16
least likely       0.61      0.74      0.67        23

    accuracy                           0.39        76
   macro avg       0.28      0.31      0.29        76
weighted avg       0.34      0.39      0.36        76



In [12]:
svc.fit(X_train, Y_train)
predictions = list(svc.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

 most likely       0.21      0.30      0.25        10
    probable       0.35      0.41      0.38        27
    possible       0.23      0.19      0.21        16
least likely       0.78      0.61      0.68        23

    accuracy                           0.41        76
   macro avg       0.39      0.38      0.38        76
weighted avg       0.44      0.41      0.42        76



In [13]:
lr.fit(X_train, Y_train)
predictions = list(lr.predict(X_test))
print(classification_report(Y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

 most likely       0.00      0.00      0.00        10
    probable       0.45      0.70      0.55        27
    possible       0.33      0.06      0.11        16
least likely       0.52      0.61      0.56        23

    accuracy                           0.45        76
   macro avg       0.33      0.34      0.30        76
weighted avg       0.39      0.45      0.39        76

